In [2]:
import numpy as np
import cvxpy as cp
import xpress as xp
import scipy.io as scio

In [3]:
data = scio.loadmat('consumer_exp/K1pi1N50.mat')
z = data['data']
price = data['price']

In [4]:
N = 50
T = 12
z = data['data'][:N]
price = data['price'][:N]
m = xp.problem()

e = [[xp.var(name="e_{0}_{1}".format(i,t), vartype=xp.continuous) for t in range(T)] for i in range(N)]
x = [[xp.var(name="x_{0}_{1}".format(i,t), vartype=xp.continuous) for t in range(T)] for i in range(N)]
u = [xp.var(name="u_{0}".format(t)) for t in range(T)]
L = [xp.var(name="L_{0}".format(t)) for t in range(T)]
M = xp.var(name="M", vartype=xp.continuous)
beta = [[xp.var(name="beta_{0}_{1}".format(i,t), vartype=xp.continuous) for t in range(T)] for i in range(N)]
alpha = [xp.var(name="alpha_{0}".format(i)) for i in range(N)]

m.addVariable(e)
m.addVariable(x)
m.addVariable(u)
m.addVariable(L)
m.addVariable(M)
m.addVariable(beta)
m.addVariable(alpha)

m.addConstraint((e[i][t] >= 0 for i in range(N) for t in range(T)),
                (x[i][t] >= 0 for i in range(N) for t in range(T)),
                (u[t]>=0 for i in range(N) for t in range(T)),
                 M >= 0,
                (L[t] >= 0 for t in range(T)),
                (beta[i][t] >= 0 for i in range(N) for t in range(T)),
                (e[i][t] >= z[i][t] - x[i][t] for i in range(N) for t in range(T)),
                (e[i][t] >= x[i][t] - z[i][t] for i in range(N) for t in range(T)),
                (xp.Sum([x[i][t] for t in range(T)]) <= M+0.2 for i in range(N)),
                (xp.Sum([x[i][t] for t in range(T)]) >= M-0.2 for i in range(N)),
                (x[i][t] <= L[t] for i in range(N) for t in range(T)),
                (alpha[i]+beta[i][t] >= u[t] - price[i][t] for i in range(N) for t in range(T)),
                (xp.Sum([u[t]*x[i][t] - price[i][t]*x[i][t] for t in range(T)]) == M*alpha[i] + xp.Sum([L[t]*beta[i][t] for t in range(T)]) for i in range(N))
                )

m.setObjective(xp.Sum([e[i][t] for i in range(N) for t in range(T)]), sense=xp.minimize)

print("status: ", m.getProbStatus())
m.nlpoptimize()

print("solution:", m.getSolution())

status:  0
Original problem size
   linear:    2488 rows, 951 columns, 4863 linear coefficients
   quadratic: 0 in obj, 25 rows, 625 in rows
Nonlinear presolve
   converted 25 quadratic matrix to SLP
   linear row reductions removed 1431 rows, 0 columns, 1649 linear coefficients
   linear duplicate row removal removed 25 rows, 0 columns, 325 linear coefficients
   eliminator removed 150 rows, 150 columns, 300 linear coefficients
   simplify removed 600 tokens
   creating 25 '+' clusters removed 550 tokens
   created 350 '*' clusters
Presolved problem size
   linear:    882 rows, 801 columns, 2589 linear coefficients
   nonlinear: 25 coefficients, 2575 tokens
Problem is nonlinear presolved
FICO Xpress v8.13.3, Community, solve started 20:49:22, Feb 3, 2022
Control settings used:
XSLP_POSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
OUTPUTLOG = 1
Maximum expanded nl-formula size: 103  (row 'R2464')
Total tokens: 2575
  12  parallel calculation threads
  Jacobian: symbolic differentiation
         